In [4]:
import geopandas as gpd
import pandas as pd
import json
import requests

## Preparação: Requisição dos dados da API do Observatório

> Nota: Não é possível solitiar vítima não fatal mais vítima fatal, pois retorna o código `http 500 - Internal Server Error` - Erro no tempo de execução. Será feito duas requisições separadas com arquivos diferentes, e então feito o merge.

In [5]:
session = requests.Session()

params_nao_fatal = {"data":"[[\"Todos\"],[\"PORTO VELHO\"],[\"VÍTIMA NÃO FATAL\"],[\"Todos\"],[\"1/2019\"],[\"12/2020\"]]",
          "labels": "[\"Natureza_do_Acidente\",\"Municipio\",\"Consequencia\",\"Via_1\",\"Data_Inicial\",\"Data_Final\"]"}

params_fatal = {"data":"[[\"Todos\"],[\"PORTO VELHO\"],[\"VÍTIMA FATAL\"],[\"Todos\"],[\"1/2019\"],[\"12/2020\"]]",
          "labels": "[\"Natureza_do_Acidente\",\"Municipio\",\"Consequencia\",\"Via_1\",\"Data_Inicial\",\"Data_Final\"]"}

cookie = {'chave':'lgpd'}

req = requests.Session.post(session, url='http://observatorio.sepog.ro.gov.br/TransitoPerfil/GetGeoData', json=params_nao_fatal, cookies=cookie, stream=True)

with open('AcidentesdeTransitoNaoFatal.json', 'wb') as arq_acidentes:
    arq_acidentes.write(req.content)

req = requests.Session.post(session, url='http://observatorio.sepog.ro.gov.br/TransitoPerfil/GetGeoData',json=params_fatal,cookies=cookie, stream=True)

with open('AcidentesdeTransitoFatal.json', 'wb') as arq_acidentes:
    arq_acidentes.write(req.content)

## Primeiro passo: Normalização do JSON
- Será normalizado usando um método do pandas: [json_normalize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html)

Formato padrão do dado:
```
{
    "<tipo de acidente>": {
          "<id_aleatorio>": {
                "LATITUDE": <float>,
                  "LONGITUDE": <float>,
                  "CONSEQUENCIA": <string>,
                  "MUNICIPIO": <string>,
                  "VEICULO_1": <string>,
                  "VEICULO_2": <string>,
                  "DATA_DO_FATO": <datetime>,
                  "FROTA": <int>
          }
    }
}
```

In [15]:
with open('AcidentesdeTransitoNaoFatal.json', 'r') as arq_acidentes:
    acidentes = json.load(arq_acidentes)

    df = pd.DataFrame() # Crio um DataFrame vazio onde irá reunir os dados normalizados

    for tipo_acidente, acidente in acidentes.items(): # Para cada tipo de acidente ...
        for id_, informacoes in acidente.items(): # Para cada item do tipo de acidente...
            df_items = pd.json_normalize(informacoes)
            df = pd.concat((df, df_items), ignore_index=True)
df

,LATITUDE,LONGITUDE,CONSEQUENCIA,MUNICIPIO,VEICULO_1,VEICULO_2,DATA_DO_FATO,FROTA
0,-8.762072,-63.843155,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,02/01/2019 00:00:00,280860
1,-8.806092,-63.884237,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,20/01/2019 00:00:00,280860
2,-8.738549,-63.862959,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,20/01/2019 00:00:00,280860
3,-8.774956,-63.892546,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,25/01/2019 00:00:00,280860
4,-8.710393,-63.985594,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,27/01/2019 00:00:00,280860
...,...,...,...,...,...,...,...,...
9553,-8.800215,-63.795577,VÍTIMA NÃO FATAL,PORTO VELHO,,,12/07/2020 00:00:00,280860
9554,-8.800483,-63.734232,VÍTIMA NÃO FATAL,PORTO VELHO,,,08/07/2020 00:00:00,280860
9555,-9.158756,-64.193839,VÍTIMA NÃO FATAL,PORTO VELHO,,,01/07/2020 00:00:00,280860
9556,-8.766685,-63.884052,VÍTIMA NÃO FATAL,PORTO VELHO,,,01/07/2020 00:00:00,280860


In [16]:
with open('AcidentesdeTransitoFatal.json', 'r') as arq_acidentes:
    acidentes = json.load(arq_acidentes)

    df2 = pd.DataFrame() # Crio um DataFrame vazio onde irá reunir os dados normalizados

    for tipo_acidente, acidente in acidentes.items(): # Para cada tipo de acidente ...
        for id_, informacoes in acidente.items(): # Para cada item do tipo de acidente...
            df_items = pd.json_normalize(informacoes)
            df2 = pd.concat((df2, df_items), ignore_index=True)

df = pd.concat((df, df2), axis=0, ignore_index=True)
df

,LATITUDE,LONGITUDE,CONSEQUENCIA,MUNICIPIO,VEICULO_1,VEICULO_2,DATA_DO_FATO,FROTA
0,-8.762072,-63.843155,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,02/01/2019 00:00:00,280860
1,-8.806092,-63.884237,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,20/01/2019 00:00:00,280860
2,-8.738549,-63.862959,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,20/01/2019 00:00:00,280860
3,-8.774956,-63.892546,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,25/01/2019 00:00:00,280860
4,-8.710393,-63.985594,VÍTIMA NÃO FATAL,PORTO VELHO,VEÍCULO ÚNICO,MOTO,27/01/2019 00:00:00,280860
...,...,...,...,...,...,...,...,...
9813,-9.347602,-64.644781,VÍTIMA FATAL,PORTO VELHO,MOTO,VEÍCULO ÚNICO,04/02/2020 00:00:00,280860
9814,-8.773548,-63.821836,VÍTIMA FATAL,PORTO VELHO,MOTO,VEÍCULO ÚNICO,04/05/2020 00:00:00,280860
9815,-9.750465,-66.618236,VÍTIMA FATAL,PORTO VELHO,CAMINHÃO,VEÍCULO ÚNICO,28/08/2019 00:00:00,280860
9816,-8.739714,-63.932140,VÍTIMA FATAL,PORTO VELHO,,,04/10/2020 00:00:00,280860


## Segundo passo: Limpeza dos dados
- Excluir dados sem LATLON
- Excluir dados sem veículo 1 ou 2 por